<pre>
<h2> Homework</h2>
<h2> CRFs를 이용한 자동 띄어쓰기 성능 향상</h2>
  입력 자질 : 직전 음절, 현재 음절, 다음 음절
  기존 방법
  ["나", "는", "사", "과", "가", "좋", "아"]
            ->
            [ {"WORD":"나"}, {"WORD":"는"}, {"WORD":"사"}, ... ]
    
    나 -> {"WORD":"나"}
    는 -> {"WORD":"는"}
    사 -> {"WORD":"사"}

    각 feature dictionary에 자질 추가
</pre>


In [1]:
!pip install sklearn-crfsuite

     |████████████████████████████████| 747kB 4.0MB/s 


In [2]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

## 1. train data, test data 구성하기

In [3]:
# 파일 경로
file_path = '/content/drive/My Drive/인공지능/week6/spacing_data.txt'

# 데이터 파일을 읽고 저장
with open(file_path, "r", encoding='utf8') as inFile:
    lines = inFile.readlines()

In [4]:
# 데이터를 음절로 이루어진 문장과 정답 값으로 나누어 저장
datas = []
for line in lines:
    pieces = line.strip().split('\t')
    eumjeol_sequence, label = pieces[0].split(), pieces[1].split()
    datas.append((eumjeol_sequence, label))

In [5]:
# train test split
num_of_train_datas = int(len(datas)*0.9)

train_datas = datas[:num_of_train_datas]
test_datas = datas[num_of_train_datas:]

print(len(train_datas))
print(len(test_datas))

900
100


## 2. 문장에서 자질 추출해 저장하기 

In [11]:
def sent2feature(eumjeol_sequence):
    before_eumjeol, after_eumjeol = "", ""
    features = []
    for index, eumjeol in enumerate(eumjeol_sequence):
      # 첫 음절일 경우 그 앞은 ^로 표시
      if index == 0:
        before_eumjeol = '^'
        after_eumjeol = eumjeol_sequence[index+1]

      # 끝 음절일 경우 그 뒤는 $로 표시
      elif index == len(eumjeol_sequence)-1:
        before_eumjeol = eumjeol_sequence[index-1]
        after_eumjeol = '$'

      # 첫 음절도 끝 음절도 아닐 경우
      else:
        before_eumjeol = eumjeol_sequence[index-1]
        after_eumjeol = eumjeol_sequence[index+1]

      feature = {"BEFORE":before_eumjeol, "CURRENT":eumjeol, "AFTER":after_eumjeol}
      features.append(feature)
    return features

In [12]:
train_x, train_y = [], []
for eumjeol_sequence, label in train_datas:
    train_x.append(sent2feature(eumjeol_sequence))
    train_y.append(label)
    
test_x, test_y = [], []
for eumjeol_sequence, label in test_datas:
    test_x.append(sent2feature(eumjeol_sequence))
    test_y.append(label)

In [18]:
train_x[0]

[{'AFTER': '속', 'BEFORE': '^', 'CURRENT': '약'},
 {'AFTER': '장', 'BEFORE': '약', 'CURRENT': '속'},
 {'AFTER': '소', 'BEFORE': '속', 'CURRENT': '장'},
 {'AFTER': '인', 'BEFORE': '장', 'CURRENT': '소'},
 {'AFTER': '신', 'BEFORE': '소', 'CURRENT': '인'},
 {'AFTER': '라', 'BEFORE': '인', 'CURRENT': '신'},
 {'AFTER': '호', 'BEFORE': '신', 'CURRENT': '라'},
 {'AFTER': '텔', 'BEFORE': '라', 'CURRENT': '호'},
 {'AFTER': '커', 'BEFORE': '호', 'CURRENT': '텔'},
 {'AFTER': '피', 'BEFORE': '텔', 'CURRENT': '커'},
 {'AFTER': '숍', 'BEFORE': '커', 'CURRENT': '피'},
 {'AFTER': '에', 'BEFORE': '피', 'CURRENT': '숍'},
 {'AFTER': '재', 'BEFORE': '숍', 'CURRENT': '에'},
 {'AFTER': '옥', 'BEFORE': '에', 'CURRENT': '재'},
 {'AFTER': '이', 'BEFORE': '재', 'CURRENT': '옥'},
 {'AFTER': '먼', 'BEFORE': '옥', 'CURRENT': '이'},
 {'AFTER': '저', 'BEFORE': '이', 'CURRENT': '먼'},
 {'AFTER': '와', 'BEFORE': '먼', 'CURRENT': '저'},
 {'AFTER': '기', 'BEFORE': '저', 'CURRENT': '와'},
 {'AFTER': '다', 'BEFORE': '와', 'CURRENT': '기'},
 {'AFTER': '리', 'BEFORE': '기', 'CURRENT'

## 3. CRFs 학습시키기

In [14]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs')
crf.fit(train_x, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=None,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=None,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

## 4. 모델 성능 평가 및 띄어쓰기 결과 출력하기

In [15]:
# 학습된 모델의 성능 평가 
predict = crf.predict(test_x)

print("Accuracy Score: " + str(metrics.flat_accuracy_score(test_y, predict)))

Accuracy Score: 0.8994658527279664


In [16]:
# 모델의 출력값을 이용한 띄어쓰기 함수
def show_predict_result(test_datas, predict):
  for index_1 in range(len(test_datas)):
    eumjeol_sequence, correct_labels = test_datas[index_1]
    predict_labels = predict[index_1]
  
    correct_sentence, predict_sentence = "", ""
    for index_2 in range(len(eumjeol_sequence)):
      # 첫 음절인 경우
      if(index_2 == 0): 
        correct_sentence += eumjeol_sequence[index_2]
        predict_sentence += eumjeol_sequence[index_2]
        continue
      
      # B태그인 음절의 경우 앞에 띄어쓰기 넣어줌
      if(correct_labels[index_2] == 'B'):
        correct_sentence += " "
      correct_sentence += eumjeol_sequence[index_2]

      # B태그인 음절의 경우 앞에 띄어쓰기 넣어줌
      if(predict_labels[index_2] == 'B'):
        predict_sentence += " "
      predict_sentence += eumjeol_sequence[index_2]

    print("정답 문장 : " + correct_sentence)
    print("출력 문장 : " + predict_sentence)
    print()
    

In [17]:
print("10개의 데이터에 대한 모델 출력과 실제 정답 비교")
print()

show_predict_result(test_datas[:10], predict[:10])

10개의 데이터에 대한 모델 출력과 실제 정답 비교

정답 문장 : 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분모였다.
출력 문장 : 19 14- 18년의 전쟁은 인류를 통합시킨 최초의 공통 분모였다.

정답 문장 : 하지만 이 전쟁은 죽음을 통해 인류를 통합시켰다.
출력 문장 : 하지만이 전쟁은 죽음을 통해 인류를 통합시켰다.

정답 문장 : 사라예보에서 한 세르비아인이 쏜 총 한발이 합스부르크가의 계승자를 죽였다.
출력 문장 : 사라 예보에서 한세르 비아인이 쏜총한 발이 합스부르크가의 계승자를 죽였다.

정답 문장 : 이 암살행위는 국지적인 민족주의들과 세계적인 제국주의들이 충돌하는 분쟁지역에서 저질러졌다.
출력 문장 : 이암 살행 위는 국지적인 민족주의 들과 세계적인 제국주의 들이 충돌하는 분쟁 지역에서 저질러졌다.

정답 문장 : 오토만제국의 점진적인 해체는 민족주의의 독기를 발산하는 동시에 오스트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.
출력 문장 : 오토만 제국의 점진적인해체는 민족주의의 독기를 발산하는 동시에 오스 트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.

정답 문장 : 이렇게 해서 발칸 반도의 한 외진 장소에서 벌어진 국지적인 테러 행위는 일련의 긴박한 반응을 불러 일으키면서 전 유럽에 영향을 미쳤을 뿐만 아니라 이번에는 아시아와 아프리카 식민지들, 일본, 그리고 이어서 미국과 멕시코까지 끌어들였다.
출력 문장 : 이렇게 해서 발칸 반도의 한외진 장소에서 벌어 진국지적인테 러행 위는 일련의 긴박한 반응을 불러일으키면서 전유럽에 영향을 미쳤을 뿐만 아니라이 번에는 아시아와 아프리 카식민지들, 일본, 그리고 이어서 미국과 멕시코까지 끌어들였다.

정답 문장 : 전쟁의 물결이 지구상의 모든 대양으로 밀려드는 동안 캐나다인들과 미국인들, 오스트레일리아인들, 세네갈인들, 알제리인들, 모로코인들, 안남(安南)인들은 연합군 깃발을 휘날리며 유럽전선에서 싸웠다.
출력 문장 : 전쟁의 물결이 지구상의 모든 대양으로 밀